In [3]:
!pip3 install -q --upgrade pip
!pip3 install -q pandas numpy matplotlib seaborn openpyxl climateserv requests netCDF4 xarray pyproj python-dotenv

In [12]:
import os
import json
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from typing import Dict, List, Tuple, Optional, Any
# Load environment variables from .env file
load_dotenv()

Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21
Python-dotenv could not parse statement starting at line 22
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24
Python-dotenv could not parse statement starting at line 25
Python-dotenv could not parse statement starting at line 26
Python-dotenv could not parse statement starting at line 27
Python-dotenv could not parse statement starting at line 35
Python-dotenv could not parse statement starting at line 37
Python-dotenv could not parse statement starting at line 38
Python-dotenv could not parse statement starting at line 39


True

In [ ]:
import requests
import time
import json
from typing import Dict, List, Tuple, Optional


def authenticate(username: str, password: str) -> str:
   """
   Authenticate with the AppEEARS API and return the authentication token.


   :param username: NASA Earthdata Login username.
   :param password: NASA Earthdata Login password.
   :return: Authentication token.
   """
   response = requests.post(
       url='https://appeears.earthdatacloud.nasa.gov/api/login',
       auth=(username, password)
   )
   return response.json()['token']


def submit_task(token: str, task_params: Dict) -> str:
   """
   Submit a task to the AppEEARS API and return the task ID.


   :param token: Authentication token.
   :param task_params: Parameters for the task.
   :return: Task ID.
   """
   response = requests.post(
       url='https://appeears.earthdatacloud.nasa.gov/api/task',
       json=task_params,
       headers={'Authorization': f'Bearer {token}'}
   )
   return response.json()['task_id']


def check_task_status(token: str, task_id: str) -> Dict:
   """
   Check the status of a task.


   :param token: Authentication token.
   :param task_id: ID of the task to check.
   :return: Status of the task.
   """
   response = requests.get(
       url=f'https://appeears.earthdatacloud.nasa.gov/api/task/{task_id}',
       headers={'Authorization': f'Bearer {token}'}
   )
   return response.json()


def create_task(long: float, lat: float, start_date: str, end_date: str, product: str, file_type: str) -> Optional[Dict]:
   """
   Create a task with the AppEEARS API.


   :param long: Longitude.
   :param lat: Latitude.
   :param start_date: Start date of the period (format MM-DD-YYYY).
   :param end_date: End date of the period (format MM-DD-YYYY).
   :param product: Product to use.
   :param file_type: Type of data file (geotiff or nc4).
   :return: Task status if task is successfully completed, otherwise None.
   """
   # User credentials (replace with actual credentials)
   username = 'your-username'
   password = 'your-password'


   # Authenticate and get token
   token = authenticate(username=username, password=password)


   # Define task parameters
   task = {
       'task_type': 'point',
       'task_name': 'AppEEARS Data Request',
       'params': {
           'dates': [{'startDate': start_date, 'endDate': end_date}],
           'layers': [{'product': product, 'layer': 'LST_Day_1km'}],
           'coordinates': [{'latitude': lat, 'longitude': long}],
           'output': {'format': {'type': file_type}}
       }
   }


   # Submit the task
   task_id = submit_task(token=token, task_params=task)
   print(f"Task submitted, ID: {task_id}")


   # Check task status periodically
   while True:
       status = check_task_status(token=token, task_id=task_id)
       if status['status'] == 'done':
           print("Task completed successfully.")
           return status
       elif status['status'] == 'error':
           print("An error occurred.")
           return None
       else:
           print(f"Task status: {status['status']}. Waiting before next check...")
           time.sleep(60)  # Wait for 60 seconds before checking again


   return None

In [17]:
def login(username: str, password: str) -> Optional[str]:
    """
    Perform a login to the AppEEARS API and retrieve the authentication token.

    Args:
    username: The NASA Earthdata Login username.
    password: The NASA Earthdata Login password.

    Returns:
    The authentication token if login is successful; None otherwise.
    """
    url = 'https://appeears.earthdatacloud.nasa.gov/api/login'
    try:
        response = requests.post(url=url, auth=(username, password))
        response.raise_for_status()
        token_info = response.json()
        return token_info.get('token')
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [18]:
def get_products(token: str, pretty: bool = True) -> Optional[Dict[str, Any]]:
    """
    Retrieve the list of available products from the AppEEARS API.

    Args:
    token: The authentication token obtained from the login function.
    pretty: Boolean to toggle pretty JSON formatting.

    Returns:
    A dictionary of available products if successful; None otherwise.
    """
    url = 'https://appeears.earthdatacloud.nasa.gov/api/product'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'pretty': pretty}
    try:
        response = requests.get(url=url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None
    
def create_dataframe(products: List[Dict[str, Any]]) -> pd.DataFrame:
    """
    Convert a list of product dictionaries into a DataFrame.

    Args:
    products: A list of dictionaries, each representing a product.

    Returns:
    A pandas DataFrame where each row represents a product.
    """
    return pd.DataFrame(products)


In [21]:
def submit_task(token: str, task_params: Dict) -> str:
    """
    Submit a task to the AppEEARS API and return the task ID.

    Args:
        token (str): Authentication token.
        task_params (Dict): Parameters for the task.

    Returns:
        str: Task ID.
    """
    response = requests.post(
        url='https://appeears.earthdatacloud.nasa.gov/api/task',
        json=task_params,
        headers={'Authorization': f'Bearer {token}'}
    )
    return response.json()['task_id']

In [22]:
def check_all_tasks_status(token: str) -> Dict:
    """
    Check the status of all tasks associated with the account.

    Args:
        token (str): Authentication token.

    Returns:
        Dict: Status of all tasks.
    """
    response = requests.get(
        url='https://appeears.earthdatacloud.nasa.gov/api/task',
        headers={'Authorization': f'Bearer {token}'}
    )
    return response.json()

In [23]:
def check_specific_task_status(token: str, task_id: str) -> Dict:
    """
    Check the status of a specific task.

    Args:
        token (str): Authentication token.
        task_id (str): ID of the task to check.

    Returns:
        Dict: Status of the task.
    """
    response = requests.get(
        url=f'https://appeears.earthdatacloud.nasa.gov/api/task/{task_id}',
        headers={'Authorization': f'Bearer {token}'}
    )
    return response.json()

In [24]:
def create_and_submit_task(username: str, password: str, long: float, lat: float, start_date: str, end_date: str, product: str, file_type: str) -> Optional[str]:
    """
    Authenticate, create, and submit a task to the AppEEARS API.

    Args:
        username (str): Username for authentication.
        password (str): Password for authentication.
        long (float): Longitude for the point task.
        lat (float): Latitude for the point task.
        start_date (str): Start date in 'MM-DD-YYYY' format.
        end_date (str): End date in 'MM-DD-YYYY' format.
        product (str): Product to use.
        file_type (str): Type of data file (geotiff or netcdf4).

    Returns:
        Optional[str]: Task ID if task is successfully submitted, None otherwise.
    """
    # Authenticate and get token
    auth_response = requests.post(
        url='https://appeears.earthdatacloud.nasa.gov/api/login',
        auth=(username, password)
    )
    if auth_response.status_code != 200:
        print(f"Authentication failed: {auth_response.json().get('message')}")
        return None

    token = auth_response.json()['token']

    # Define task parameters
    task_params = {
        'task_type': 'point',
        'task_name': 'AppEEARS Data Request',
        'params': {
            'dates': [{'startDate': start_date, 'endDate': end_date}],
            'layers': [{'product': product, 'layer': 'LST_Day_1km'}],  # Adjust the layer as needed
            'coordinates': [{'latitude': lat, 'longitude': long}],
            'output': {'format': {'type': file_type}}
        }
    }

    # Submit the task
    task_id = submit_task(token=token, task_params=task_params)
    return task_id

In [25]:
def create_and_submit_ndvi_task(username: str, password: str, long: float, lat: float, start_date: str, end_date: str, product: str, layer: str, file_type: str) -> Optional[str]:
    """
    Authenticate, create, and submit an NDVI task to the AppEEARS API.

    Args:
        username (str): Username for authentication.
        password (str): Password for authentication.
        long (float): Longitude for the point task.
        lat (float): Latitude for the point task.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        product (str): Product to use.
        layer (str): The specific layer of the product to use.
        file_type (str): Type of data file (geotiff or netcdf4).

    Returns:
        Optional[str]: Task ID if task is successfully submitted, None otherwise.
    """
    # Authenticate and get token
    auth_response = requests.post(
        url='https://appeears.earthdatacloud.nasa.gov/api/login',
        auth=(username, password)
    )
    if auth_response.status_code != 200:
        print(f"Authentication failed: {auth_response.json().get('message')}")
        return None

    token = auth_response.json()['token']

    # Define task parameters
    task_params = {
        'task_type': 'point',
        'task_name': 'NDVI Data Request',
        'params': {
            'dates': [{'startDate': start_date, 'endDate': end_date}],
            'layers': [{'product': product, 'layer': layer}],
            'coordinates': [{'latitude': lat, 'longitude': long}],
            'output': {'format': {'type': file_type}}
        }
    }

    # Submit the task
    task_id = submit_task(token=token, task_params=task_params)
    return task_id

In [19]:
# Example Usage
username = os.getenv('EARTHDATA_USERNAME')
password = os.getenv('EARTHDATA_PASSWORD')

# Login and get token
token = login(username=username, password=password)

if token:
    # Get products
    products = get_products(token=token)
    df = create_dataframe(products=products)

In [26]:
# Example usage with your specified parameters
username = os.getenv('EARTHDATA_USERNAME')
password = os.getenv('EARTHDATA_PASSWORD')
longitude = -16.716666  # Your specified longitude
latitude = 13.42080389502958  # Your specified latitude
start_date = '2015-01-01'  # Start date of the period
end_date = '2022-12-31'    # End date of the period
product = 'MOD13Q1.061'    # Your specified product
layer = '250m_16_days_NDVI'  # Your specified layer from the image
file_type = 'netcdf4'  

task_id = create_and_submit_ndvi_task(
    username=username, 
    password=password, 
    long=longitude, 
    lat=latitude, 
    start_date=start_date, 
    end_date=end_date, 
    product=product,
    layer=layer,
    file_type=file_type
)

if task_id:
    print(f"NDVI task submitted successfully. Task ID: {task_id}")
else:
    print("Failed to submit NDVI task.")

Failed to submit NDVI task.


In [27]:
import requests

def authenticate(username: str, password: str) -> Optional[str]:
    """Authenticate with the AppEEARS API to get a token."""
    auth_response = requests.post(
        'https://appeears.earthdatacloud.nasa.gov/api/login',
        auth=(username, password)
    )
    if auth_response.status_code == 200:
        return auth_response.json()['token']
    else:
        print(f"Authentication failed with status code {auth_response.status_code}: {auth_response.text}")
        return None

def submit_task(token: str, task_params: dict) -> Optional[str]:
    """Submit a task to the AppEEARS API."""
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.post(
        'https://appeears.earthdatacloud.nasa.gov/api/task',
        json=task_params,
        headers=headers
    )
    if response.status_code == 202:
        return response.json()['task_id']
    else:
        print(f"Task submission failed with status code {response.status_code}: {response.text}")
        return None

def create_and_submit_ndvi_task(username: str, password: str, long: float, lat: float, start_date: str, end_date: str, product: str, layer: str, file_type: str) -> Optional[str]:
    """Create and submit an NDVI data request task to AppEEARS."""
    token = authenticate(username=username, password=password)
    if token is None:
        return None

    task_params = {
        'task_type': 'point',
        'task_name': 'NDVI Data Request',
        'params': {
            'dates': [{'startDate': start_date, 'endDate': end_date}],
            'layers': [{'product': product, 'layer': layer}],
            'coordinates': [{'latitude': lat, 'longitude': long}],
            'output': {'format': {'type': file_type}}
        }
    }

    return submit_task(token=token, task_params=task_params)

In [30]:
# Replace with your actual username and password
username = os.getenv('EARTHDATA_USERNAME')
password = os.getenv('EARTHDATA_PASSWORD')

# Specify the latitude, longitude, start date, end date, product, layer, and file type
longitude = -16.716666
latitude = 13.42080389502958
start_date = '01-01-2015'
end_date = '12-31-2022'
product = 'MOD13Q1'
layer = '250m_16_days_NDVI'
file_type = 'geotiff'

# Create and submit the task
task_id = create_and_submit_ndvi_task(
    username=username, 
    password=password, 
    long=longitude, 
    lat=latitude, 
    start_date=start_date, 
    end_date=end_date, 
    product=product,
    layer=layer,
    file_type=file_type
)

if task_id:
    print(f"NDVI task submitted successfully. Task ID: {task_id}")
else:
    print("Failed to submit NDVI task.")

Task submission failed with status code 400: {"message": "Product MOD13Q1 is not available"}

Failed to submit NDVI task.


In [31]:
df

,Product,Platform,Description,RasterType,Resolution,TemporalGranularity,Version,Available,DocLink,Source,TemporalExtentStart,TemporalExtentEnd,Deleted,DOI,Info,ProductAndVersion
0,GPW_DataQualityInd,GPW,Quality of Input Data for Population Count and...,Tile,1000m,Quinquennial,411,True,https://doi.org/10.7927/H42Z13KG,SEDAC,2000-01-01,2020-12-31,False,10.7927/H42Z13KG,{},GPW_DataQualityInd.411
1,GPW_UN_Adj_PopCount,GPW,UN-adjusted Population Count,Tile,1000m,Quinquennial,411,True,https://doi.org/10.7927/H4PN93PB,SEDAC,2000-01-01,2020-12-31,False,10.7927/H4PN93PB,{},GPW_UN_Adj_PopCount.411
2,GPW_UN_Adj_PopDensity,GPW,UN-adjusted Population Density,Tile,1000m,Quinquennial,411,True,https://doi.org/10.7927/H4F47M65,SEDAC,2000-01-01,2020-12-31,False,10.7927/H4F47M65,{},GPW_UN_Adj_PopDensity.411
3,GPW_Basic_Demog_Char,GPW,Basic Demographic Characteristics - Population...,Tile,1000m,Static,411,True,https://doi.org/10.7927/H46M34XX,SEDAC,2010-01-01,2010-01-01,False,10.7927/H46M34XX,{},GPW_Basic_Demog_Char.411
4,MCD12Q1,Combined MODIS,Land Cover Type,Tile,500m,Yearly,061,True,https://doi.org/10.5067/MODIS/MCD12Q1.061,LP DAAC,2001-01-01,2020-12-31,False,10.5067/MODIS/MCD12Q1.061,{},MCD12Q1.061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,HLSS30,Harmonized Landsat and Sentinel-2,Land Surface Reflectance,Tile,30m,Daily,020,True,https://doi.org/10.5067/HLS/HLSS30.002,LP DAAC,2015-12-01,Present,False,10.5067/HLS/HLSS30.002,"{'provider_id': 'lp_prod', 'is_utm_zonal': Tru...",HLSS30.020
116,HLSL30,Harmonized Landsat and Sentinel-2,Land Surface Reflectance,Tile,30m,Daily,020,True,https://doi.org/10.5067/HLS/HLSL30.002,LP DAAC,2013-04-01,Present,False,10.5067/HLS/HLSL30.002,"{'provider_id': 'lp_prod', 'is_utm_zonal': Tru...",HLSL30.020
117,GEOLST4KHR,MEaSUREs LSTE,Geostationary Earth Orbit Land Surface Tempera...,Swath,4000m,Hourly,002,True,https://doi.org/10.5067/MEaSUREs/LSTE/GEOLST4K...,LP DAAC,2000-04-01,2016-12-31,False,10.5067/MEaSUREs/LSTE/GEOLST4KHR.002,{},GEOLST4KHR.002
118,WaterBalance_Daily_Historical_GRIDMET,"NPS Historical Water Balance, Daily",NPS Water Balance,Tile,1000m,Daily,015,True,https://doi.org/10.1002/ecs2.4530,ORNL,1979-01-01,2021-12-31,False,110.1002/ecs2.4530,{'edc_provider_id': 'other'},WaterBalance_Daily_Historical_GRIDMET.015
